In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [14]:
OPENAI_API_KEY = 'sk-proj-zfM7I0DbwVOSrCqSjsYFT3BlbkFJsCcbfFZaDNxMdYF1DwKn'

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.5)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

chain = prompt | model

In [37]:
from operator import itemgetter
from typing import List

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableFieldSpec,
    RunnablePassthrough,
)
from langchain_core.runnables.history import RunnableWithMessageHistory


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [38]:
print(store)

{}


In [24]:
store2 = {}

def get_by_session_id_test(session_id: str) -> BaseChatMessageHistory:
    
    if session_id not in store2:
        store2[session_id] = session_id #InMemoryHistory()
    return store2[session_id]

In [39]:

chain_with_history = RunnableWithMessageHistory(
    chain,
    # Uses the get_by_session_id function defined in the example
    # above.
    get_by_session_id,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [41]:
chain_with_history.invoke({'input':'quero ajuda para escrever um código?'},config={"configurable": {"session_id": "foo"}})

AIMessage(content='Claro, ficarei feliz em ajudar! Qual é a linguagem de programação que você está usando e qual é o objetivo do código que você precisa escrever? Por favor, me forneça mais detalhes para que eu possa te auxiliar da melhor forma possível.', response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 98, 'total_tokens': 157}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a95c19cf-0825-4bba-8e70-d392df1ab4f0-0')

In [52]:
type(store['foo'])

__main__.InMemoryHistory